In [1]:
# CELL 1: Install dependencies
import sys, subprocess
pkgs = ["pandas", "numpy", "plotly", "ipywidgets", "kaleido"]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + pkgs + ["--quiet"])
print("Done!")

Done!


In [2]:
# CELL 2: Imports
import os
import warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

warnings.filterwarnings('ignore')

print("="*70)
print("📊 ML Model Comparison Dashboard")
print("Compare models from the same split method")
print("="*70)

📊 ML Model Comparison Dashboard
Compare models from the same split method


In [3]:
# CELL 3: Folder Browser Widget

class FolderBrowser:
    """Interactive folder browser widget."""
    
    def __init__(self, start='.', label='Folder'):
        self.cur = Path(start).resolve()
        self.sel = self.cur
        self.label = label
        
        self.html = widgets.HTML(f"<code>{self.cur}</code>")
        self.dd = widgets.Select(
            options=self._list(), 
            layout=widgets.Layout(width='100%', height='150px')
        )
        self.b_up = widgets.Button(description='↑ Up', layout=widgets.Layout(width='70px'))
        self.b_in = widgets.Button(description='→ Enter', layout=widgets.Layout(width='80px'))
        self.b_sel = widgets.Button(description='✓ Select', button_style='success', 
                                    layout=widgets.Layout(width='80px'))
        self.selhtml = widgets.HTML(f"<b>Selected:</b> <code>{self.sel}</code>")
        
        self.b_up.on_click(lambda b: self._up())
        self.b_in.on_click(lambda b: self._enter())
        self.b_sel.on_click(lambda b: self._select())
        
        self.w = widgets.VBox([
            widgets.HTML(f"<b>{label}</b>"),
            self.html, self.dd,
            widgets.HBox([self.b_up, self.b_in, self.b_sel]),
            self.selhtml
        ])
    
    def _list(self):
        try:
            items = ['.']
            for x in sorted(self.cur.iterdir()):
                if x.is_dir() and not x.name.startswith('.'):
                    items.append(f"📁 {x.name}")
            return items
        except:
            return ['.']
    
    def _refresh(self):
        self.html.value = f"<code>{self.cur}</code>"
        self.dd.options = self._list()
    
    def _up(self):
        if self.cur.parent != self.cur:
            self.cur = self.cur.parent
            self._refresh()
    
    def _enter(self):
        if self.dd.value and self.dd.value.startswith('📁'):
            name = self.dd.value[2:].strip()
            p = self.cur / name
            if p.is_dir():
                self.cur = p
                self._refresh()
    
    def _select(self):
        self.sel = self.cur
        self.selhtml.value = f"<b>Selected:</b> <code>{self.sel}</code>"
    
    def path(self):
        return self.sel


print("Folder browser loaded.")

Folder browser loaded.


In [4]:
# CELL 4: Model Results Loader

def load_model_results(method_folder):
    """
    Load all model results from a method folder.
    
    Expected structure:
    method_folder/
        ├── ModelName1/
        │   ├── *_metrics.csv
        │   ├── *_predictions.csv
        │   ├── *_feature_importance.csv
        │   └── *_summary.txt
        ├── ModelName2/
        └── ...
    """
    method_folder = Path(method_folder)
    results = {}
    
    # Detect method name from folder
    method_name = method_folder.name
    
    # Find all model subfolders
    model_folders = [f for f in method_folder.iterdir() if f.is_dir()]
    
    print(f"\n📂 Loading results from: {method_folder}")
    print(f"   Method: {method_name}")
    print(f"   Found {len(model_folders)} model folders")
    
    for model_folder in sorted(model_folders):
        model_name = model_folder.name
        model_data = {
            'name': model_name,
            'folder': model_folder,
            'metrics': None,
            'predictions': None,
            'feature_importance': None,
        }
        
        # Find and load metrics CSV
        metrics_files = list(model_folder.glob('*_metrics.csv'))
        if metrics_files:
            model_data['metrics'] = pd.read_csv(metrics_files[0])
        
        # Find and load predictions CSV
        pred_files = list(model_folder.glob('*_predictions.csv'))
        if pred_files:
            model_data['predictions'] = pd.read_csv(pred_files[0])
        
        # Find and load feature importance CSV
        fi_files = list(model_folder.glob('*_feature_importance.csv'))
        if fi_files:
            model_data['feature_importance'] = pd.read_csv(fi_files[0])
        
        # Check if we have at least metrics
        if model_data['metrics'] is not None:
            results[model_name] = model_data
            print(f"   ✓ {model_name}: metrics loaded")
        else:
            print(f"   ⚠ {model_name}: no metrics found, skipping")
    
    return results, method_name


def build_comparison_dataframe(results):
    """
    Build a unified DataFrame for comparison.
    """
    rows = []
    
    for model_name, data in results.items():
        metrics_df = data['metrics']
        
        # Extract metrics for train and test
        train_metrics = metrics_df[metrics_df['split'] == 'train'].set_index('metric')['value'].to_dict()
        test_metrics = metrics_df[metrics_df['split'] == 'test'].set_index('metric')['value'].to_dict()
        
        row = {
            'Model': model_name,
            'R²_train': train_metrics.get('R²', np.nan),
            'R²_test': test_metrics.get('R²', np.nan),
            'RMSE_train': train_metrics.get('RMSE', np.nan),
            'RMSE_test': test_metrics.get('RMSE', np.nan),
            'MAE_train': train_metrics.get('MAE', np.nan),
            'MAE_test': test_metrics.get('MAE', np.nan),
            'ExpVar_train': train_metrics.get('Explained Var', np.nan),
            'ExpVar_test': test_metrics.get('Explained Var', np.nan),
        }
        
        # Calculate gaps (overfitting indicators)
        row['R²_gap'] = row['R²_train'] - row['R²_test']
        row['RMSE_gap'] = row['RMSE_test'] - row['RMSE_train']
        row['MAE_gap'] = row['MAE_test'] - row['MAE_train']
        
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    # Sort by test R² (best first)
    df = df.sort_values('R²_test', ascending=False).reset_index(drop=True)
    df['Rank'] = df.index + 1
    
    return df


print("Model loader functions defined.")

Model loader functions defined.


In [5]:
# CELL 5: Visualization Functions

def create_r2_comparison_figure(df, method_name):
    """
    Create R² comparison bar chart (train vs test for each model).
    """
    fig = go.Figure()
    
    # Sort by test R²
    df_sorted = df.sort_values('R²_test', ascending=True)
    
    # Train bars
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['R²_train'],
        name='Train R²',
        orientation='h',
        marker_color='#3498db',
        text=[f"{v:.4f}" for v in df_sorted['R²_train']],
        textposition='inside',
    ))
    
    # Test bars
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['R²_test'],
        name='Test R²',
        orientation='h',
        marker_color='#e74c3c',
        text=[f"{v:.4f}" for v in df_sorted['R²_test']],
        textposition='inside',
    ))
    
    fig.update_layout(
        title=f"<b>R² Comparison</b><br><sup>Method: {method_name} | Higher is better</sup>",
        xaxis_title='R² Score',
        yaxis_title='Model',
        barmode='group',
        height=max(400, len(df) * 50),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        xaxis=dict(range=[0, 1.05])
    )
    
    return fig


def create_error_comparison_figure(df, method_name):
    """
    Create RMSE and MAE comparison.
    """
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('RMSE (lower is better)', 'MAE (lower is better)'),
        horizontal_spacing=0.15
    )
    
    # Sort by test RMSE
    df_sorted = df.sort_values('RMSE_test', ascending=False)
    
    # RMSE
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['RMSE_train'],
        name='Train RMSE',
        orientation='h',
        marker_color='#3498db',
        showlegend=True,
    ), row=1, col=1)
    
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['RMSE_test'],
        name='Test RMSE',
        orientation='h',
        marker_color='#e74c3c',
        showlegend=True,
    ), row=1, col=1)
    
    # MAE
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['MAE_train'],
        name='Train MAE',
        orientation='h',
        marker_color='#2ecc71',
        showlegend=True,
    ), row=1, col=2)
    
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['MAE_test'],
        name='Test MAE',
        orientation='h',
        marker_color='#e67e22',
        showlegend=True,
    ), row=1, col=2)
    
    fig.update_layout(
        title=f"<b>Error Metrics Comparison</b><br><sup>Method: {method_name}</sup>",
        barmode='group',
        height=max(400, len(df) * 50),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
    )
    
    return fig


def create_overfitting_figure(df, method_name):
    """
    Create overfitting analysis figure (train-test gap).
    """
    fig = go.Figure()
    
    # Sort by R² gap (most overfitting first)
    df_sorted = df.sort_values('R²_gap', ascending=True)
    
    # Color based on overfitting level
    colors = []
    for gap in df_sorted['R²_gap']:
        if gap > 0.15:
            colors.append('#e74c3c')  # Red - high overfitting
        elif gap > 0.05:
            colors.append('#f39c12')  # Orange - moderate
        else:
            colors.append('#27ae60')  # Green - good
    
    fig.add_trace(go.Bar(
        y=df_sorted['Model'],
        x=df_sorted['R²_gap'],
        orientation='h',
        marker_color=colors,
        text=[f"{v:.4f}" for v in df_sorted['R²_gap']],
        textposition='outside',
    ))
    
    # Add threshold lines
    fig.add_vline(x=0.05, line_dash='dash', line_color='orange', 
                  annotation_text='Moderate', annotation_position='top')
    fig.add_vline(x=0.15, line_dash='dash', line_color='red',
                  annotation_text='High', annotation_position='top')
    
    fig.update_layout(
        title=f"<b>Overfitting Analysis (R² Gap: Train - Test)</b><br><sup>Method: {method_name} | Lower is better (green < 0.05 < orange < 0.15 < red)</sup>",
        xaxis_title='R² Gap (Train - Test)',
        yaxis_title='Model',
        height=max(400, len(df) * 50),
    )
    
    return fig


def create_radar_comparison_figure(df, method_name):
    """
    Create radar chart comparing models on multiple metrics.
    """
    # Normalize metrics for radar (0-1 scale, higher is better)
    df_norm = df.copy()
    
    # R² already 0-1, higher is better
    df_norm['R²_norm'] = df_norm['R²_test']
    
    # RMSE: invert (lower is better -> higher is better)
    rmse_max = df_norm['RMSE_test'].max()
    df_norm['RMSE_norm'] = 1 - (df_norm['RMSE_test'] / rmse_max) if rmse_max > 0 else 0
    
    # MAE: invert
    mae_max = df_norm['MAE_test'].max()
    df_norm['MAE_norm'] = 1 - (df_norm['MAE_test'] / mae_max) if mae_max > 0 else 0
    
    # Generalization: 1 - gap (lower gap is better)
    gap_max = df_norm['R²_gap'].max()
    df_norm['Gen_norm'] = 1 - (df_norm['R²_gap'] / gap_max) if gap_max > 0 else 1
    
    categories = ['R² (test)', 'Low RMSE', 'Low MAE', 'Generalization']
    
    fig = go.Figure()
    
    colors = px.colors.qualitative.Set2
    
    for i, (_, row) in enumerate(df_norm.iterrows()):
        values = [row['R²_norm'], row['RMSE_norm'], row['MAE_norm'], row['Gen_norm']]
        values.append(values[0])  # Close the radar
        
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories + [categories[0]],
            fill='toself',
            name=row['Model'],
            line_color=colors[i % len(colors)],
            opacity=0.6,
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 1])
        ),
        title=f"<b>Multi-Metric Radar Comparison</b><br><sup>Method: {method_name} | Larger area = better model</sup>",
        height=600,
        showlegend=True,
    )
    
    return fig


def create_predictions_scatter(results, method_name):
    """
    Create combined scatter plot of predictions for all models.
    """
    fig = go.Figure()
    
    colors = px.colors.qualitative.Set2
    
    all_obs = []
    all_pred = []
    
    for i, (model_name, data) in enumerate(results.items()):
        if data['predictions'] is not None:
            pred_df = data['predictions']
            test_data = pred_df[pred_df['split'] == 'test']
            
            if len(test_data) > 0:
                obs = test_data['observed'].values
                pred = test_data['predicted'].values
                
                all_obs.extend(obs)
                all_pred.extend(pred)
                
                # Subsample if too many points
                if len(obs) > 500:
                    idx = np.random.choice(len(obs), 500, replace=False)
                    obs = obs[idx]
                    pred = pred[idx]
                
                fig.add_trace(go.Scatter(
                    x=obs,
                    y=pred,
                    mode='markers',
                    name=model_name,
                    marker=dict(
                        color=colors[i % len(colors)],
                        size=6,
                        opacity=0.5
                    ),
                    hovertemplate=f"{model_name}<br>Obs: %{{x:.2f}}<br>Pred: %{{y:.2f}}<extra></extra>"
                ))
    
    # Add 1:1 line
    if all_obs:
        min_val = min(min(all_obs), min(all_pred))
        max_val = max(max(all_obs), max(all_pred))
        fig.add_trace(go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            name='1:1 line',
            line=dict(color='black', dash='dash', width=2)
        ))
    
    fig.update_layout(
        title=f"<b>Test Predictions: All Models</b><br><sup>Method: {method_name}</sup>",
        xaxis_title='Observed',
        yaxis_title='Predicted',
        height=600,
        legend=dict(orientation='v', yanchor='top', y=0.99, xanchor='left', x=1.02),
    )
    
    return fig


def create_feature_importance_heatmap(results, method_name):
    """
    Create heatmap of feature importance across models.
    """
    # Collect all feature importances
    importance_data = {}
    all_features = set()
    
    for model_name, data in results.items():
        if data['feature_importance'] is not None:
            fi_df = data['feature_importance']
            # Get gain importance (most common)
            gain_fi = fi_df[fi_df['type'] == 'gain']
            if len(gain_fi) > 0:
                importance_data[model_name] = dict(zip(gain_fi['feature'], gain_fi['importance']))
                all_features.update(gain_fi['feature'].tolist())
    
    if not importance_data:
        return None
    
    # Build matrix
    all_features = sorted(all_features)
    models = list(importance_data.keys())
    
    matrix = []
    for feature in all_features:
        row = []
        for model in models:
            row.append(importance_data[model].get(feature, 0))
        matrix.append(row)
    
    # Normalize per model (column)
    matrix = np.array(matrix)
    col_max = matrix.max(axis=0, keepdims=True)
    col_max[col_max == 0] = 1
    matrix_norm = matrix / col_max
    
    fig = go.Figure(data=go.Heatmap(
        z=matrix_norm,
        x=models,
        y=all_features,
        colorscale='Blues',
        text=[[f"{v:.4f}" for v in row] for row in matrix],
        texttemplate="%{text}",
        textfont={"size": 8},
        hovertemplate="Feature: %{y}<br>Model: %{x}<br>Importance: %{text}<extra></extra>"
    ))
    
    fig.update_layout(
        title=f"<b>Feature Importance Heatmap (Gain)</b><br><sup>Method: {method_name} | Normalized per model</sup>",
        xaxis_title='Model',
        yaxis_title='Feature',
        height=max(400, len(all_features) * 25),
    )
    
    return fig


def create_ranking_table_figure(df, method_name):
    """
    Create a visual ranking table.
    """
    # Prepare data
    df_display = df[['Rank', 'Model', 'R²_test', 'R²_train', 'R²_gap', 'RMSE_test', 'MAE_test']].copy()
    
    # Format values
    df_display['R²_test'] = df_display['R²_test'].apply(lambda x: f"{x:.4f}")
    df_display['R²_train'] = df_display['R²_train'].apply(lambda x: f"{x:.4f}")
    df_display['R²_gap'] = df_display['R²_gap'].apply(lambda x: f"{x:.4f}")
    df_display['RMSE_test'] = df_display['RMSE_test'].apply(lambda x: f"{x:.4f}")
    df_display['MAE_test'] = df_display['MAE_test'].apply(lambda x: f"{x:.4f}")
    
    # Color rows based on rank
    colors = []
    for rank in df_display['Rank']:
        if rank == 1:
            colors.append('#d4edda')  # Green - best
        elif rank == 2:
            colors.append('#fff3cd')  # Yellow - 2nd
        elif rank == 3:
            colors.append('#ffeeba')  # Light yellow - 3rd
        else:
            colors.append('#ffffff')  # White
    
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Rank</b>', '<b>Model</b>', '<b>R² Test</b>', '<b>R² Train</b>', 
                    '<b>R² Gap</b>', '<b>RMSE Test</b>', '<b>MAE Test</b>'],
            fill_color='#2c3e50',
            font=dict(color='white', size=12),
            align='center',
            height=35
        ),
        cells=dict(
            values=[df_display[col] for col in df_display.columns],
            fill_color=[colors],
            align='center',
            font=dict(size=11),
            height=30
        )
    )])
    
    fig.update_layout(
        title=f"<b>🏆 Model Ranking</b><br><sup>Method: {method_name} | Sorted by Test R²</sup>",
        height=max(300, len(df) * 35 + 100)
    )
    
    return fig


print("Visualization functions defined.")

Visualization functions defined.


In [6]:
# CELL 6: Main Comparison Dashboard

class ModelComparisonDashboard:
    def __init__(self):
        self.results = None
        self.method_name = None
        self.comparison_df = None
        self.figures = {}
        
        self._build_ui()
    
    def _build_ui(self):
        style = {'description_width': 'initial'}
        
        # Folder browser
        self.folder_browser = FolderBrowser('.', label='📁 Select Method Folder (max_time_dist, random, or overfit)')
        
        # Load button
        self.load_btn = widgets.Button(
            description='📂 Load Results',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.load_btn.on_click(self._load_results)
        
        # Info display
        self.info_html = widgets.HTML("<i>Select a method folder and click 'Load Results'</i>")
        
        # Output directory for saving
        self.save_browser = FolderBrowser('.', label='💾 Save Figures To')
        self.save_btn = widgets.Button(
            description='💾 Save All Figures',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.save_btn.on_click(self._save_figures)
        
        # Outputs
        self.log_output = widgets.Output(
            layout=widgets.Layout(border='1px solid #ccc', max_height='200px', overflow='auto')
        )
        self.plot_output = widgets.Output(
            layout=widgets.Layout(border='1px solid #ccc', min_height='600px')
        )
        
        # Build UI
        self.ui = widgets.VBox([
            widgets.HTML("""
                <div style="background: linear-gradient(135deg, #2c3e50 0%, #27ae60 100%); 
                            padding: 15px; border-radius: 8px; margin-bottom: 15px;">
                    <h2 style="color: white; margin: 0;">📊 ML Model Comparison Dashboard</h2>
                    <p style="color: #ecf0f1; margin: 5px 0 0 0;">Compare models from the same split method</p>
                </div>
            """),
            widgets.HTML('''
                <div style="background:#fff3cd; padding:10px; border-radius:5px; margin:10px 0;">
                <b>📋 Instructions:</b><br>
                <small>
                1. Navigate to a method folder (e.g., <code>max_time_dist/</code>, <code>random/</code>, or <code>overfit/</code>)<br>
                2. Click "Load Results" to read all model subfolders<br>
                3. View comparison visualizations<br>
                4. Optionally save figures to a folder<br>
                <b>Note:</b> Compare models within the SAME method only (not across methods)
                </small>
                </div>
            '''),
            self.folder_browser.w,
            self.load_btn,
            self.info_html,
            widgets.HTML('<hr>'),
            widgets.HTML('<b>Log:</b>'),
            self.log_output,
            widgets.HTML('<hr>'),
            widgets.HBox([self.save_browser.w, widgets.VBox([widgets.HTML('<br>'), self.save_btn])]),
            widgets.HTML('<hr>'),
            widgets.HTML('<b>📈 Comparison Results:</b>'),
            self.plot_output
        ])
    
    def _load_results(self, b):
        """Load results from the selected folder."""
        with self.log_output:
            clear_output()
            
            folder = self.folder_browser.path()
            print(f"Loading from: {folder}")
            
            try:
                self.results, self.method_name = load_model_results(folder)
                
                if not self.results:
                    self.info_html.value = "<span style='color:red'>❌ No valid model results found!</span>"
                    return
                
                # Build comparison DataFrame
                self.comparison_df = build_comparison_dataframe(self.results)
                
                self.info_html.value = f"""
                    <div style="background:#d4edda; padding:10px; border-radius:5px;">
                        <b>✅ Loaded {len(self.results)} models</b><br>
                        Method: <code>{self.method_name}</code><br>
                        Models: {', '.join(self.results.keys())}
                    </div>
                """
                
                # Generate visualizations
                self._generate_visualizations()
                
            except Exception as e:
                self.info_html.value = f"<span style='color:red'>❌ Error: {e}</span>"
                import traceback
                traceback.print_exc()
    
    def _generate_visualizations(self):
        """Generate all comparison visualizations."""
        with self.plot_output:
            clear_output()
            
            df = self.comparison_df
            method = self.method_name
            
            print("Generating visualizations...")
            
            # 1. Ranking Table
            print("  📊 Creating ranking table...")
            fig_ranking = create_ranking_table_figure(df, method)
            fig_ranking.show()
            self.figures['ranking'] = fig_ranking
            
            # 2. R² Comparison
            print("  📊 Creating R² comparison...")
            fig_r2 = create_r2_comparison_figure(df, method)
            fig_r2.show()
            self.figures['r2_comparison'] = fig_r2
            
            # 3. Error Comparison
            print("  📊 Creating error comparison...")
            fig_error = create_error_comparison_figure(df, method)
            fig_error.show()
            self.figures['error_comparison'] = fig_error
            
            # 4. Overfitting Analysis
            print("  📊 Creating overfitting analysis...")
            fig_overfit = create_overfitting_figure(df, method)
            fig_overfit.show()
            self.figures['overfitting'] = fig_overfit
            
            # 5. Radar Comparison
            print("  📊 Creating radar comparison...")
            fig_radar = create_radar_comparison_figure(df, method)
            fig_radar.show()
            self.figures['radar'] = fig_radar
            
            # 6. Predictions Scatter
            print("  📊 Creating predictions scatter...")
            fig_scatter = create_predictions_scatter(self.results, method)
            fig_scatter.show()
            self.figures['predictions'] = fig_scatter
            
            # 7. Feature Importance Heatmap
            print("  📊 Creating feature importance heatmap...")
            fig_heatmap = create_feature_importance_heatmap(self.results, method)
            if fig_heatmap:
                fig_heatmap.show()
                self.figures['importance_heatmap'] = fig_heatmap
            else:
                print("    ⚠ No feature importance data available")
            
            print("\n✅ All visualizations generated!")
            
            # Print summary
            print("\n" + "="*60)
            print("📋 SUMMARY")
            print("="*60)
            best_model = df.iloc[0]
            print(f"🏆 Best Model: {best_model['Model']}")
            print(f"   R² (test): {best_model['R²_test']:.4f}")
            print(f"   R² gap: {best_model['R²_gap']:.4f}")
            print(f"   RMSE (test): {best_model['RMSE_test']:.4f}")
    
    def _save_figures(self, b):
        """Save all figures to the selected folder."""
        if not self.figures:
            with self.log_output:
                print("❌ No figures to save! Load results first.")
            return
        
        save_dir = self.save_browser.path()
        
        with self.log_output:
            print(f"\n💾 Saving figures to: {save_dir}")
            
            for name, fig in self.figures.items():
                try:
                    filename = f"comparison_{self.method_name}_{name}.png"
                    filepath = save_dir / filename
                    fig.write_image(str(filepath), scale=2)
                    print(f"   ✓ Saved: {filename}")
                except Exception as e:
                    print(f"   ⚠ Failed to save {name}: {e}")
            
            # Also save comparison CSV
            try:
                csv_file = save_dir / f"comparison_{self.method_name}_summary.csv"
                self.comparison_df.to_csv(csv_file, index=False)
                print(f"   ✓ Saved: {csv_file.name}")
            except Exception as e:
                print(f"   ⚠ Failed to save CSV: {e}")
            
            print("\n✅ All files saved!")
    
    def show(self):
        display(self.ui)


print("Model Comparison Dashboard class defined.")

Model Comparison Dashboard class defined.


In [7]:
# CELL 7: Run the dashboard

print("="*70)
print("📊 ML Model Comparison Dashboard")
print("="*70)
print("")
print("Expected folder structure:")
print("  {method}/              ← Select this folder")
print("    ├── ModelName1/")
print("    │   ├── *_metrics.csv")
print("    │   ├── *_predictions.csv")
print("    │   └── *_feature_importance.csv")
print("    ├── ModelName2/")
print("    └── ...")
print("")
print("Visualizations generated:")
print("  🏆 Ranking Table - sorted by test R²")
print("  📊 R² Comparison - train vs test bars")
print("  📉 Error Comparison - RMSE and MAE")
print("  ⚠️ Overfitting Analysis - R² gap")
print("  🎯 Radar Chart - multi-metric comparison")
print("  📈 Predictions Scatter - all models overlay")
print("  🔥 Feature Importance Heatmap")
print("")

dashboard = ModelComparisonDashboard()
dashboard.show()

📊 ML Model Comparison Dashboard

Expected folder structure:
  {method}/              ← Select this folder
    ├── ModelName1/
    │   ├── *_metrics.csv
    │   ├── *_predictions.csv
    │   └── *_feature_importance.csv
    ├── ModelName2/
    └── ...

Visualizations generated:
  🏆 Ranking Table - sorted by test R²
  📊 R² Comparison - train vs test bars
  📉 Error Comparison - RMSE and MAE
  ⚠️ Overfitting Analysis - R² gap
  🎯 Radar Chart - multi-metric comparison
  📈 Predictions Scatter - all models overlay
  🔥 Feature Importance Heatmap

